In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost 

In [0]:
import pandas as pd
import numpy as np


from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/Matrix/dw_matrix-/matrix_two'

/content/drive/My Drive/Colab Notebooks/Matrix/dw_matrix-/matrix_two


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values


In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [ x for x in cat_feats if 'price' not in x]

In [0]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-19747.81093847179, 75.95212671816309)

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

DecisionTree

In [0]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19747.81093847179, 75.95212671816309)

RandomForest

In [0]:
run_model(RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0), cat_feats)

In [0]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[21:02:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:03:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:03:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13034.249470063352, 103.73566521784606)

In [0]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

In [0]:
feats= [
      'param_napęd__cat',
      'param_rok-produkcji__cat',
      'param_stan__cat',
      'param_skrzynia-biegów__cat',
      'param_faktura-vat__cat',
      'param_moc__cat',
      'param_marka-pojazdu__cat',
      'feature_kamera-cofania__cat',
      'param_typ__cat',
      'param_pojemność-skokowa__cat',
      'seller_name__cat',
      'feature_wspomaganie-kierownicy__cat',
      'param_model-pojazdu__cat',
      'param_wersja__cat',
      'param_kod-silnika__cat',
      'feature_system-start-stop__cat',
      'feature_asystent-pasa-ruchu__cat',
      'feature_czujniki-parkowania-przednie__cat',
      'feature_łopatki-zmiany-biegów__cat',
      'feature_regulowane-zawieszenie__cat'
]


In [0]:
len(feats)

20

In [0]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[21:05:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:05:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:06:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13475.642177294301, 72.42283542993323)

In [0]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [0]:
df['param_rok-produkcji']= df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [0]:
feats= [
      'param_napęd__cat',
      'param_rok-produkcji',
      'param_stan__cat',
      'param_skrzynia-biegów__cat',
      'param_faktura-vat__cat',
      'param_moc__cat',
      'param_marka-pojazdu__cat',
      'feature_kamera-cofania__cat',
      'param_typ__cat',
      'param_pojemność-skokowa__cat',
      'seller_name__cat',
      'feature_wspomaganie-kierownicy__cat',
      'param_model-pojazdu__cat',
      'param_wersja__cat',
      'param_kod-silnika__cat',
      'feature_system-start-stop__cat',
      'feature_asystent-pasa-ruchu__cat',
      'feature_czujniki-parkowania-przednie__cat',
      'feature_łopatki-zmiany-biegów__cat',
      'feature_regulowane-zawieszenie__cat'
]

In [0]:
run_model(xgb.XGBRegressor(**xgb_params), feats )

[21:06:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:06:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:06:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11327.392744425772, 66.22128239389319)

In [0]:
 df['param_moc']= df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]) )

In [0]:
feats= [
      'param_napęd__cat',
      'param_rok-produkcji',
      'param_stan__cat',
      'param_skrzynia-biegów__cat',
      'param_faktura-vat__cat',
      'param_moc',
      'param_marka-pojazdu__cat',
      'feature_kamera-cofania__cat',
      'param_typ__cat',
      'param_pojemność-skokowa__cat',
      'seller_name__cat',
      'feature_wspomaganie-kierownicy__cat',
      'param_model-pojazdu__cat',
      'param_wersja__cat',
      'param_kod-silnika__cat',
      'feature_system-start-stop__cat',
      'feature_asystent-pasa-ruchu__cat',
      'feature_czujniki-parkowania-przednie__cat',
      'feature_łopatki-zmiany-biegów__cat',
      'feature_regulowane-zawieszenie__cat'
]

In [0]:
run_model(xgb.XGBRegressor(**xgb_params), feats )

[21:06:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:06:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:06:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9743.019185925381, 71.23754505718838)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')) )

In [0]:
feats= [
      'param_napęd__cat',
      'param_rok-produkcji',
      'param_stan__cat',
      'param_skrzynia-biegów__cat',
      'param_faktura-vat__cat',
      'param_moc',
      'param_marka-pojazdu__cat',
      'feature_kamera-cofania__cat',
      'param_typ__cat',
      'param_pojemność-skokowa',
      'seller_name__cat',
      'feature_wspomaganie-kierownicy__cat',
      'param_model-pojazdu__cat',
      'param_wersja__cat',
      'param_kod-silnika__cat',
      'feature_system-start-stop__cat',
      'feature_asystent-pasa-ruchu__cat',
      'feature_czujniki-parkowania-przednie__cat',
      'feature_łopatki-zmiany-biegów__cat',
      'feature_regulowane-zawieszenie__cat'
]

In [0]:
run_model(xgb.XGBRegressor(**xgb_params), feats )

[21:06:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:07:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:07:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9570.015830397702, 53.097008450207305)